In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
%cd '/content/drive/My Drive/Data Science/'

/content/drive/My Drive/Data Science


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
import pandas as pd
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB

In [0]:
data_file = "11152019_train_final.csv"
df_train = pd.read_csv(data_file)

In [0]:
test_file = "11152019_test_final.csv"
df_test = pd.read_csv(test_file)

In [0]:
y_train = df_train['new.returningCustomer']
X_train = df_train.drop(columns=['new.returningCustomer'])

y_test = df_test['new.returningCustomer']
X_test = df_test.drop(columns=['new.returningCustomer']) 

In [0]:
X_train.dtypes

In [0]:
X_test.dtypes

## Converting Categorical Data to Numerical

In [0]:
def label_encoder(X):

  lb = preprocessing.LabelBinarizer()
  X['trafficSource.isTrueDirect'] = lb.fit_transform(X['trafficSource.isTrueDirect'])
  X['device.isMobile'] = lb.transform(X['device.isMobile'])

  ignoreColumns = {''}
  for column in X.columns:
      if (X[column].dtype != np.int64 and X[column].dtype != np.float64):
          print(f"Converting column: {column}")

          le = preprocessing.LabelEncoder()
          X[column] = le.fit_transform(X[column])
          del le

  return X       

In [0]:
X_train = label_encoder(X_train)
X_test = label_encoder(X_test)

In [0]:
X_train.dtypes

In [0]:
X_test.dtypes

## Feature Selection

In [12]:
model = ExtraTreesClassifier()
model.fit(X_train, y_train)
# display the relative importance of each attribute
importances = model.feature_importances_
indices = np.argsort(importances)[::-1]

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [0]:
unimp_col = []
for f in range(X_train.shape[1]):
    print("%d. feature %d %s (%f)" % (f + 1, indices[f], X_train.columns[indices[f]], importances[indices[f]]))
    if(importances[f]) < 0.01:
      unimp_col += [X_train.columns[f]]

unimp_col      

In [0]:
keep_cols = ['totals.bounces',
             'totals.hits',
             'totals.newVisits',
             'totals.pageviews',
             'totals.sessionQualityDim',
             'totals.timeOnSite',
             'trafficSource.isTrueDirect',
             'visitNumber',
             'country_dem',
             'latitude',
             'longitude']

In [0]:
unimp_col = []
for col in X_train.columns:
  if col not in keep_cols:
    unimp_col += [col]

41

In [38]:
X_train_imp = X_train.drop(columns = unimp_col)
print(X_train_imp.shape)

X_test_imp = X_test.drop(columns = unimp_col)
X_test_imp.shape

(1537503, 11)


(361429, 11)

## Normalization of Data

In [0]:
def ss_scale(X):
  scaler = preprocessing.StandardScaler()
  X[X.columns] = scaler.fit_transform(X[X.columns])
  return X

In [0]:
def mm_scale(X):
  scaler = preprocessing.MinMaxScaler()
  X[X.columns] = scaler.fit_transform(X[X.columns])
  return X

In [0]:
X_train_ss = ss_scale(X_train_imp)
X_test_ss = ss_scale(X_test_imp)

X_train_mm = mm_scale(X_train_imp)
X_test_mm = mm_scale(X_test_imp)

## Linear Regression

In [0]:
regressor = LinearRegression()
regressor.fit(X_train_ss, y_train)

y_pred_lr = regressor.predict(X_test_ss)

In [43]:
df_result_lr = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred_lr})
total_lr = df_result_lr.shape[0]
df_result_lr

,Actual,Predicted
0,1,1.014561
1,1,1.012108
2,1,0.999533
3,1,1.001931
4,0,0.174451
...,...,...
361424,0,0.174445
361425,0,0.139098
361426,1,0.169994
361427,1,0.956381


In [44]:
correct_lr = df_result_lr[abs(df_result_lr['Actual'] - df_result_lr['Predicted']) < 0.25].shape[0]
acc_lr = correct_lr/total_lr * 100
acc_lr

84.76021569934898

In [45]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred_lr))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred_lr))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred_lr)))

Mean Absolute Error: 0.21366084728591292
Mean Squared Error: 0.11321787462389155
Root Mean Squared Error: 0.33647863917920784


## Bayesian

In [46]:
 clf = GaussianNB()
 clf.fit(X_train_ss, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [0]:
y_pred_gnb = clf.predict(X_test_ss)

In [48]:
clf.score(X_test_ss, y_test)

0.8472452404206635

In [49]:
df_result_gnb = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred_gnb})
total_gnb = df_result_gnb.shape[0]
df_result_gnb

,Actual,Predicted
0,1,1
1,1,1
2,1,1
3,1,1
4,0,0
...,...,...
361424,0,0
361425,0,0
361426,1,0
361427,1,1


In [50]:
correct_gnb = df_result_gnb[abs(df_result_gnb['Actual'] - df_result_gnb['Predicted']) < 0.25].shape[0]
acc_gnb = correct_gnb/total_gnb * 100
acc_gnb

84.72452404206635

In [51]:
 clf = MultinomialNB()
 clf.fit(X_train_mm, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [0]:
y_pred_mnb = clf.predict(X_test_mm)

In [53]:
clf.score(X_test_mm, y_test)

0.8010480619983454

In [54]:
df_result_mnb = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred_mnb})
total_mnb = df_result_mnb.shape[0]
df_result_mnb

,Actual,Predicted
0,1,1
1,1,1
2,1,1
3,1,1
4,0,0
...,...,...
361424,0,0
361425,0,0
361426,1,0
361427,1,1


In [55]:
correct_mnb = df_result_mnb[abs(df_result_mnb['Actual'] - df_result_mnb['Predicted']) < 0.25].shape[0]
acc_mnb = correct_mnb/total_mnb * 100
acc_mnb

80.10480619983454

# Clustering

In [0]:
kmeans = KMeans(n_clusters=2, random_state=0).fit(X_train_mm)

In [57]:
kmeans.labels_

array([1, 1, 0, ..., 0, 0, 0], dtype=int32)

In [0]:
y_pred_cl = kmeans.predict(X_test_mm)

In [59]:
kmeans.score(X_test_mm)

-258786.10792106332

In [60]:
kmeans.cluster_centers_

array([[5.28411454e-01, 5.83593756e-03, 9.74188973e-01, 4.64423355e-03,
        1.23360976e-02, 5.29891102e-03, 5.90877347e-12, 1.36369810e-04,
        3.65631653e-01, 6.43646562e-01, 4.68305912e-01],
       [4.73936165e-01, 8.92027756e-03, 3.54539576e-01, 6.90586935e-03,
        3.26131202e-02, 8.91228812e-03, 9.28086786e-01, 8.42862905e-03,
        6.07784776e-01, 6.68416920e-01, 3.67009880e-01]])

In [62]:
df_result_cl = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred_cl})
total_cl = df_result_cl.shape[0]
df_result_cl

,Actual,Predicted
0,1,1
1,1,1
2,1,1
3,1,1
4,0,0
...,...,...
361424,0,0
361425,0,0
361426,1,0
361427,1,1


In [64]:
correct_cl = df_result_cl[abs(df_result_cl['Actual'] - df_result_cl['Predicted']) < 0.25].shape[0]
acc_cl = correct_cl/total_cl * 100
acc_cl

76.20832860672498